# Fine-tuning RadBert #
Full parameter fine tuning of RadBERT model (RadBERT-RoBERTa-4m model from the RadBERT paper). Fine tuning carried out on the multi-label muti-class classification of reports, where each report can have multiple labels (For ex, a report can havel label consolidation-right and consolidation-2)

In [1]:
import os
import time, datetime
import codecs
from itertools import product

import random
import numpy as np
import pandas as pd

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

import transformers
from transformers import AutoTokenizer, AutoModel
from transformers import pipeline

/home/users/pranav.rao/miniconda3/envs/TutorialCuda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for usage information.


/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for usage information.




In [2]:
## Seeding everything for reproducibility
def seed_everything(seed: int):    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [3]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.width', 1000)

torch.set_printoptions(linewidth=200)

In [4]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [5]:
device='cuda:0'

## Forward pass ##
Implementing RadBERTMultiClassMulti Label PyTorch Model

In [6]:
class RadBERTMultiClassMultiLabel(nn.Module):
    """
    RadBERTMultiClassMultiLabel: Model expects batches of natural language sentences, will
    classify reports with multiple label
    """
    def __init__(self, num_classes, checkpoint, device):
        super().__init__()
        self.num_classes = num_classes
        self.checkpoint = checkpoint
        self.device = device

        self.tokenizer = AutoTokenizer.from_pretrained(self.checkpoint)
        self.transformer_encoder = AutoModel.from_pretrained(self.checkpoint)
        self.transformer_encoder_hidden_size = self.transformer_encoder.config.hidden_size
        self.linear_classifier = nn.Linear(self.transformer_encoder_hidden_size, self.num_classes)
    
    def forward(self, x):
        tokenized_inp = self.tokenizer(x, padding=True, truncation=True, return_tensors='pt').to(self.device)
        encoder_out = self.transformer_encoder(**tokenized_inp)
        logits = self.linear_classifier(encoder_out.last_hidden_state[:, 0, :])
        return logits


In [7]:
checkpoint = 'UCSD-VA-health/RadBERT-RoBERTa-4m'
#labels_subset = "normal tuberculosis opacity bronchialdilation density parenchymalopacity ett aorticenlargement mediastinalwidening mediastinalmass\
#        copd prominentbronchovascularmarkings bronchitis markings vascularprominence interval interstitiallungdisease bluntedcp effusion cardiomegaly\
#        consolidation subtle_normal peffusion lineandtube thickening haziness hilarprominence hilar inhomogenousopacity rotation\
#        calcification unfoldedaorta bandlikeopacity aorticcalcification aorticknucklecalcification fibrosis suture cardiacshift degenspine nodule\
#        pneumonia inspiration fracture pneumonitis justfibrosis lesion nonaorticcalcification tuberculosispure pleuralthickening feedingtube".split()
labels_subset = [e.strip() for e in open('/home/users/pranav.rao/MiniTasks/Radbert/labels_top50_withloc.txt', 'r').readlines()]
num_classes = len(labels_subset)

In [ ]:
radbert_multi_model = RadBERTMultiClassMultiLabel(num_classes, checkpoint, device).to(device)

In [ ]:
print(radbert_multi_model)
print(list(map(lambda x : x.shape, radbert_multi_model.parameters())))

## Custom Loss function ##
Custom loss function for multi class multi label classification to handle uncertain tags (tags have value 0 -> absent, 1 -> present, -100 -> uncertain)

In [80]:
class MultiClassMultiLabel(nn.Module):
    def __init__(self, uncertain_label, device):
        super(MultiClassMultiLabel, self).__init__()
        self.uncertain_label = uncertain_label
        self.device = device
    
    def forward(self, output, target, penalize_certainity=False):
        certain_mask = (target != self.uncertain_label)
        loss_func = nn.MultiLabelSoftMarginLoss(weight=certain_mask.type(torch.float))
        if not penalize_certainity:
            return loss_func(output, target)
        else:
            certainity_loss_func = nn.MultiLabelSoftMarginLoss(weight=(~certain_mask).type(torch.float))
            ones = torch.ones(output.shape, device=self.device)
            zeros = torch.zeros(output.shape, device=self.device)
            return loss_func(output, target) + 0.1 * (certainity_loss_func(output, ones) + certainity_loss_func(output, zeros))

In [81]:
multiclass_multilabel_loss = MultiClassMultiLabel(-100, device).to(device)

### Testing MultiClassMultiLabel loss function ###

In [82]:
logit_tensor = torch.Tensor([[-1.0, 2.0, 1.0, 5.0, -3.0], [4.0, -2.0, 1.0, -1.0, 2.5]]).to(device)
target_tensor_act = torch.Tensor([[0, 1, -100, 0, 0], [1, -100, 0, 0, 1]]).to(device)

In [83]:
#certain_mask = (target_tensor_act != -100)
#print(certain_mask.type(torch.float))
#loss_func = nn.MultiLabelSoftMarginLoss(weight=(certain_mask).type(torch.float))
#print(loss_func(logit_tensor, target_tensor_act))
print(multiclass_multilabel_loss(logit_tensor, target_tensor_act))
print(multiclass_multilabel_loss(logit_tensor, target_tensor_act, penalize_certainity=True))

tensor(0.7219, device='cuda:0')
tensor(0.7607, device='cuda:0')


In [84]:
import math
from math import log, exp

def log_sigmoid(x):
    return log(1/(1+exp(-1*x)))

In [85]:
loss1 = -1 * (1.0 * log_sigmoid(1.0) + 1.0 * log_sigmoid(2.0) + 0.0 * log_sigmoid(-1.0) + 1.0 * log_sigmoid(-5.0) + 1.0 * log_sigmoid(3.0)) / 5.0
loss2 = -1 * (1.0 * log_sigmoid(4.0) + 0.0 * log_sigmoid(2.0) + 1.0 * log_sigmoid(-1.0) + 1.0 * log_sigmoid(1.0) + 1.0 * log_sigmoid(2.5)) / 5.0

print(loss1)
print(loss2)
print((loss1 + loss2)/2.0)

1.0990984797248111
0.34471260744936094
0.721905543587086


In [86]:
loss1 = -1 * (1.0 * log_sigmoid(1.0) + 1.0 * log_sigmoid(2.0) + 0.1 * (log_sigmoid(-1.0) + log_sigmoid(1.0)) + 1.0 * log_sigmoid(-5.0) + 1.0 * log_sigmoid(3.0)) / 5.0
loss2 = -1 * (1.0 * log_sigmoid(4.0) + 0.1 * (log_sigmoid(2.0) + log_sigmoid(-2.0)) + 1.0 * log_sigmoid(-1.0) + 1.0 * log_sigmoid(1.0) + 1.0 * log_sigmoid(2.5)) / 5.0

print(loss1)
print(loss2)
print((loss1 + loss2)/2.0)

1.13162894722554
0.3897897278910799
0.76070933755831


## Custom DataLoader and Dataset ##
Read csv file, get the report from path and prepare the data

In [87]:
class ReportTagsDataset(Dataset):
    def __init__(self, tags_csv_file, report_base_path, labels_subset=None, text_transform=None, target_transform=None):
        self.report_base_path = report_base_path
        self.tags_csv_file = tags_csv_file

        self.tags_df = pd.read_csv(self.tags_csv_file)
        self.column_names = list(self.tags_df.columns.values)
        self.column_names[0] = 'filename'
        self.tags_df.columns = self.column_names

        self.labels_subset = labels_subset
        self.text_transform = text_transform
        self.target_transform = target_transform
    
    def __len__(self):
        return self.tags_df.shape[0]
    
    def __getitem__(self, index):
        report_path = os.path.join(self.report_base_path, self.tags_df.iloc[index, 0].split('/')[-1] + '.txt')
        #report_text = open(report_path).read()
        report_text = codecs.open(report_path, 'r', encoding='utf-8', errors='ignore').read()
        if self.labels_subset is None:
            target_list = torch.Tensor(list(self.tags_df.iloc[index][1:]))
        else:
            target_list = torch.Tensor(list(self.tags_df[self.labels_subset].iloc[index]))
        return report_text, target_list

In [88]:
#report_base_path = "/models_common_e2e/cxr_data/reports/training"
train_reports_base_path = '/home/users/pranav.rao/MiniTasks/Radbert/data/train'
test_reports_base_path = '/home/users/pranav.rao/MiniTasks/Radbert/data/test'
train_tags_file = '/home/users/pranav.rao/Downloads/report_tags_25k_train.csv'
test_tags_file = '/home/users/pranav.rao/Downloads/report_tags_25k_test.csv'

In [89]:
train_data = ReportTagsDataset(train_tags_file, train_reports_base_path, labels_subset=labels_subset)
test_data = ReportTagsDataset(test_tags_file, test_reports_base_path, labels_subset=labels_subset)

In [90]:
#train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
#test_dataloader = DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)
train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=True)

### Testing Dataset and DataLoader ###

In [91]:
# Display image and label.
train_features, train_labels = next(iter(train_dataloader))

In [92]:
print(train_features)
print(train_labels)
print(f"Feature batch shape: {len(train_features)}")
print(f"Labels batch shape: {len(train_labels)}")
report_text = train_features[0]
print(report_text)
label = train_labels[0]
print(f"Label: {label}")

('6016667|909308|Investigation: X-Ray Chest AP Supine (Portable)\nResults:\nArtifact +.\nSubtle haze seen in left mid and lower zones.  \nCardiac size and pulmonary vasculature cannot be assessed (AP view).\nVisualised bones appear normal.\nAdvise: Clinical correlation. \n6016667|909308|Investigation: X-Ray Chest AP Supine (Portable)\nResults:\nArtifact +.\nSubtle haze seen in left mid and lower zones.  \nCardiac size and pulmonary vasculature cannot be assessed (AP view).\nVisualised bones appear normal.\nAdvise: Clinical correlation. ', '6229228|4281617|X-Ray Chest PA/AP View of 22-FEB-2018:\nResults:\nNo focal lesion seen in the lung parenchyma. \nCP angles and domes of the diaphragm are normal.\nBoth hila are normal. \nCardiac size and configuration is normal.\nTrachea is central; no mediastinal shift is seen.\nBony thorax and soft tissues of the chest wall are normal.\n6229228|4281617|X-Ray Chest PA/AP View of 22-FEB-2018:\nResults:\nNo focal lesion seen in the lung parenchyma. \n

In [93]:
with torch.no_grad():
    logit_tensor = radbert_multi_model(train_features)

In [94]:
print(logit_tensor)
print(logit_tensor.shape)

tensor([[-2.8738, -0.3806, -0.2766,  ..., -3.9745, -4.2900, -2.7723],
        [ 0.6204, -3.2360, -2.9979,  ..., -4.6585, -4.4890, -3.9174],
        [-3.9546, -0.1608,  0.2215,  ..., -3.7216, -3.9332, -2.7011],
        ...,
        [-3.4354, -0.9878, -0.4057,  ..., -4.1158, -4.3209, -3.2361],
        [ 1.0332, -3.2712, -3.2869,  ..., -4.5339, -4.3962, -3.9428],
        [ 0.8829, -3.2148, -3.2063,  ..., -4.5850, -4.4622, -3.9426]], device='cuda:0')
torch.Size([32, 76])


## Fine Tuning ##
Fine tuning the RadBERT model for tags prediction task

### Adam Optimizer ###
Using Adam optimizer with learning rate 3e-5, beta1 = 0.9, beta2 = 0.99, l2 weight decay of 0.01

In [ ]:
lr = 3e-5
beta1 = 0.9
beta2 = 0.99
l2_weight_decay = 0.01

In [ ]:
optimizer = torch.optim.Adam(radbert_multi_model.parameters(), lr=lr, betas=(beta1, beta2), weight_decay=l2_weight_decay)

In [ ]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    for i, data in enumerate(train_dataloader):
        inputs, labels = data
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = radbert_multi_model(inputs)

        # Compute the loss and its gradients
        loss = multiclass_multilabel_loss(outputs, labels)
        loss.backward()
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 50 == 49:
            last_loss = running_loss / 50 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(train_dataloader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

### The full fine-tuning loog ###

In [ ]:
total_epochs = 5

In [ ]:
# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))

for epoch in range(total_epochs):
    # Make sure gradient tracking is on, and do a pass over the data
    print('EPOCH {}:'.format(epoch + 1))
    radbert_multi_model.train(True)
    avg_loss = train_one_epoch(epoch, writer)
    model_path = '/home/users/pranav.rao/MiniTasks/Radbert/ModelPool/model_{}_{}'.format(timestamp, epoch)
    torch.save(radbert_multi_model.state_dict(), model_path)

    # Set the model to evaluation mode, disabling dropout and using population, statistics for batch normalization
    running_vloss = 0.0
    radbert_multi_model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(test_dataloader):
            vinputs, vlabels = vdata
            vlabels = vlabels.to(device)
            voutputs = radbert_multi_model(vinputs)
            vloss = multiclass_multilabel_loss(voutputs, vlabels)
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch + 1)
    writer.flush()

## Analyzing fine-tuned models ##

In [9]:
sentence_list = ["The report shows small right-sided pleural effusion", "The report shows small left-sided pleural effusion",\
    "The report shows large right-sided pleural effusion", "The report shows large left-sided pleural effusion",\
    "There are no abnormalities in the report",\
    "There is severe consolidation in the left side","There is severe consolidation in the right side",\
    "There is mild consolidation in the right side", "There is mild consolidation in the left side"
]

sentence1_base = "A <SizeModifier> <AbnormalReport> can be seen in the report in the <LocationModifier> part"
sentence2_base = "The report shows a <SizeModifier> <LocationModifier> <AbnormalReport>"
size_modifiers = ['small', 'large']
loc_modifiers = ['upper-left', 'lower-left', 'right-sided', 'left-sided']
abnormal_report = ['pleural effusion']

l1 = [sentence1_base.replace('<SizeModifier>', size_mod).replace('<LocationModifier>', loc_mod).replace('<AbnormalReport>', ab_rep) for size_mod, loc_mod, ab_rep in product(size_modifiers, loc_modifiers, abnormal_report)]
l2 = [sentence2_base.replace('<SizeModifier>', size_mod).replace('<LocationModifier>', loc_mod).replace('<AbnormalReport>', ab_rep) for size_mod, loc_mod, ab_rep in product(size_modifiers, loc_modifiers, abnormal_report)]

negative_sentences = ['The report shows no pleural effusion', 'The report shows no consolidation on any side']
all_sentence_list = l1 + l2 + negative_sentences + sentence_list[4:]

In [10]:
print('\n'.join(all_sentence_list))

A small pleural effusion can be seen in the report in the upper-left part
A small pleural effusion can be seen in the report in the lower-left part
A small pleural effusion can be seen in the report in the right-sided part
A small pleural effusion can be seen in the report in the left-sided part
A large pleural effusion can be seen in the report in the upper-left part
A large pleural effusion can be seen in the report in the lower-left part
A large pleural effusion can be seen in the report in the right-sided part
A large pleural effusion can be seen in the report in the left-sided part
The report shows a small upper-left pleural effusion
The report shows a small lower-left pleural effusion
The report shows a small right-sided pleural effusion
The report shows a small left-sided pleural effusion
The report shows a large upper-left pleural effusion
The report shows a large lower-left pleural effusion
The report shows a large right-sided pleural effusion
The report shows a large left-sid

In [8]:
def get_sentence_embeddings(model, input):
    with torch.no_grad():
        tokenized_inp = model.tokenizer(input, padding=True, truncation=True, return_tensors='pt').to(device)
        encoder_out = model.transformer_encoder(**tokenized_inp)
    return encoder_out.last_hidden_state[:, 0, :]

In [9]:
def calc_cosine_sim_matrix(sentence_embeddings):
    cosine_sim_matrix = F.cosine_similarity(sentence_embeddings.unsqueeze(1), sentence_embeddings.unsqueeze(0), dim=2)
    return cosine_sim_matrix.cpu().detach().numpy()

In [10]:
model_paths = ['/home/users/pranav.rao/MiniTasks/Radbert/ModelPool/model_20230921_122506_0', '/home/users/pranav.rao/MiniTasks/Radbert/ModelPool/model_20230921_122506_1', '/home/users/pranav.rao/MiniTasks/Radbert/ModelPool/model_20230921_122506_2',\
              '/home/users/pranav.rao/MiniTasks/Radbert/ModelPool/model_20231011_125912_0', '/home/users/pranav.rao/MiniTasks/Radbert/ModelPool/model_20231011_125912_1', '/home/users/pranav.rao/MiniTasks/Radbert/ModelPool/model_20231011_125912_2']

radbert_multi_models = [RadBERTMultiClassMultiLabel(num_classes, checkpoint, device).to(device) for _ in range(len(model_paths))]
for i in range(len(model_paths)):
    radbert_multi_models[i].load_state_dict(torch.load(model_paths[i]))

Some weights of RobertaModel were not initialized from the model checkpoint at UCSD-VA-health/RadBERT-RoBERTa-4m and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at UCSD-VA-health/RadBERT-RoBERTa-4m and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at UCSD-VA-health/RadBERT-RoBERTa-4m and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model

In [14]:
all_embeddings_list = [get_sentence_embeddings(radbert_multi_model, all_sentence_list) for radbert_multi_model in radbert_multi_models]

In [15]:
print(all_embeddings_list)
print('\t'.join([str(all_embeddings.shape) for all_embeddings in all_embeddings_list]))

[tensor([[-0.2118, -0.4423, -0.0757,  ..., -2.4072, -0.4965,  0.2453],
        [-0.2125, -0.4571, -0.0629,  ..., -2.4554, -0.4707,  0.2653],
        [-0.1845, -0.4634, -0.0398,  ..., -2.4514, -0.4513,  0.1721],
        ...,
        [-0.0389, -0.5642, -0.2998,  ..., -2.4936, -0.4983, -0.2105],
        [-0.0359, -0.4579, -0.3213,  ..., -2.5296, -0.4663, -0.3121],
        [-0.0329, -0.4840, -0.3144,  ..., -2.4801, -0.4135, -0.3127]], device='cuda:0'), tensor([[-0.1108, -0.3812, -0.1214,  ..., -2.2235, -0.2562,  0.7130],
        [-0.1015, -0.3882, -0.1133,  ..., -2.2259, -0.2494,  0.7169],
        [-0.1083, -0.4010, -0.1160,  ..., -2.2430, -0.2553,  0.7141],
        ...,
        [ 0.0140, -0.4092, -0.1766,  ..., -2.4137, -0.6614, -0.0468],
        [ 0.0556, -0.3672, -0.1809,  ..., -2.4128, -0.6022, -0.0776],
        [ 0.0566, -0.3561, -0.1883,  ..., -2.3904, -0.5925, -0.0630]], device='cuda:0'), tensor([[-0.0670, -0.5211, -0.2080,  ..., -1.9040, -0.0530,  0.1121],
        [-0.0676, -0.5314

In [16]:
cosine_sim_list = [calc_cosine_sim_matrix(all_embeddings) for all_embeddings in all_embeddings_list]

In [17]:
pd.DataFrame(cosine_sim_list[0])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,1.000000,0.998686,0.996926,0.997273,0.996819,0.996826,0.993533,0.993887,0.988073,0.986422,0.982926,0.983263,0.984408,0.984454,0.979357,0.979540,0.855266,0.689137,0.486096,0.909067,0.906196,0.908305,0.911528
1,0.998686,1.000000,0.997085,0.997428,0.994291,0.996902,0.992408,0.992779,0.986184,0.987673,0.982474,0.982829,0.981306,0.984337,0.977526,0.977762,0.858583,0.693755,0.491522,0.907179,0.904372,0.908011,0.911084
2,0.996926,0.997085,1.000000,0.999333,0.993928,0.995342,0.996742,0.996062,0.986359,0.986547,0.988059,0.987331,0.982872,0.984633,0.984356,0.983496,0.858753,0.696377,0.492757,0.911232,0.909853,0.911084,0.912798
3,0.997273,0.997428,0.999333,1.000000,0.994278,0.995693,0.996067,0.996751,0.986633,0.986835,0.987368,0.988090,0.983197,0.984997,0.983727,0.984327,0.859006,0.694348,0.492618,0.910352,0.906987,0.908749,0.912337
4,0.996819,0.994291,0.993928,0.994278,1.000000,0.998686,0.996954,0.997303,0.986159,0.983041,0.981204,0.981609,0.988720,0.987284,0.983743,0.984007,0.849077,0.684360,0.483433,0.916214,0.913039,0.906779,0.910209
5,0.996826,0.996902,0.995342,0.995693,0.998686,1.000000,0.996980,0.997355,0.985421,0.985396,0.981822,0.982237,0.986632,0.988180,0.982855,0.983165,0.853329,0.689654,0.488803,0.914953,0.911831,0.907249,0.910546
6,0.993533,0.992408,0.996742,0.996067,0.996954,0.996980,1.000000,0.999297,0.984167,0.982796,0.986033,0.985344,0.986966,0.987164,0.988544,0.987727,0.851719,0.690588,0.489198,0.918171,0.916527,0.909330,0.911232
7,0.993887,0.992779,0.996062,0.996751,0.997303,0.997355,0.999297,1.000000,0.984426,0.983096,0.985301,0.986104,0.987274,0.987540,0.987863,0.988559,0.851912,0.688484,0.488908,0.917301,0.913602,0.906997,0.910819
8,0.988073,0.986184,0.986359,0.986633,0.986159,0.985421,0.984167,0.984426,1.000000,0.998009,0.995618,0.995883,0.996766,0.996481,0.992484,0.992580,0.877661,0.703160,0.500637,0.906327,0.903467,0.901087,0.904371
9,0.986422,0.987673,0.986547,0.986835,0.983041,0.985396,0.982796,0.983096,0.998009,1.000000,0.995467,0.995811,0.993291,0.996782,0.990640,0.990866,0.881732,0.708548,0.506824,0.904577,0.901812,0.901246,0.904318


In [18]:
pd.DataFrame(cosine_sim_list[1])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,1.000000,0.999904,0.999716,0.999765,0.999849,0.999830,0.999630,0.999682,0.997567,0.997647,0.997429,0.997213,0.997183,0.997337,0.997111,0.996912,0.992365,0.654073,0.531146,0.687346,0.683620,0.681153,0.684451
1,0.999904,1.000000,0.999660,0.999739,0.999674,0.999846,0.999494,0.999576,0.997262,0.997556,0.997168,0.996983,0.996791,0.997160,0.996760,0.996595,0.992006,0.653383,0.531467,0.686198,0.682442,0.680006,0.683329
2,0.999716,0.999660,1.000000,0.999946,0.999496,0.999514,0.999845,0.999794,0.997082,0.997198,0.997582,0.997236,0.996619,0.996804,0.997183,0.996854,0.992152,0.659063,0.535147,0.691484,0.687903,0.685423,0.688584
3,0.999765,0.999739,0.999946,1.000000,0.999542,0.999590,0.999786,0.999844,0.997312,0.997464,0.997693,0.997479,0.996839,0.997061,0.997284,0.997087,0.992449,0.657345,0.534737,0.689987,0.686244,0.683821,0.687140
4,0.999849,0.999674,0.999496,0.999542,1.000000,0.999904,0.999717,0.999767,0.997598,0.997601,0.997395,0.997173,0.997569,0.997648,0.997437,0.997234,0.992486,0.652987,0.528729,0.687342,0.683629,0.680879,0.684159
5,0.999830,0.999846,0.999514,0.999590,0.999904,1.000000,0.999656,0.999737,0.997378,0.997595,0.997214,0.997025,0.997264,0.997559,0.997170,0.997001,0.992205,0.652240,0.528909,0.686176,0.682432,0.679706,0.683012
6,0.999630,0.999494,0.999845,0.999786,0.999717,0.999656,1.000000,0.999946,0.997180,0.997218,0.997613,0.997261,0.997076,0.997186,0.997580,0.997245,0.992339,0.657980,0.532721,0.691498,0.687931,0.685168,0.688310
7,0.999682,0.999576,0.999794,0.999844,0.999767,0.999737,0.999946,1.000000,0.997411,0.997485,0.997725,0.997505,0.997299,0.997446,0.997683,0.997481,0.992636,0.656275,0.532295,0.690019,0.686290,0.683581,0.686882
8,0.997567,0.997262,0.997082,0.997312,0.997598,0.997378,0.997180,0.997411,1.000000,0.999878,0.999634,0.999703,0.999787,0.999745,0.999490,0.999572,0.996205,0.640977,0.524106,0.676492,0.672392,0.669701,0.673351
9,0.997647,0.997556,0.997198,0.997464,0.997601,0.997595,0.997218,0.997485,0.999878,1.000000,0.999555,0.999662,0.999579,0.999784,0.999324,0.999447,0.995949,0.639952,0.523548,0.675252,0.671118,0.668428,0.672105


In [19]:
pd.DataFrame(cosine_sim_list[2])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,1.000000,0.999984,0.999952,0.999954,0.999986,0.999969,0.999964,0.999961,0.998347,0.998392,0.998220,0.998111,0.998382,0.998428,0.998287,0.998169,0.994179,0.596586,0.689358,0.605031,0.603126,0.601359,0.603238
1,0.999984,1.000000,0.999934,0.999937,0.999971,0.999986,0.999946,0.999945,0.998237,0.998315,0.998109,0.997999,0.998272,0.998351,0.998177,0.998058,0.993981,0.595995,0.688972,0.604482,0.602585,0.600810,0.602682
2,0.999952,0.999934,1.000000,0.999989,0.999912,0.999892,0.999986,0.999971,0.998316,0.998359,0.998295,0.998171,0.998319,0.998361,0.998329,0.998198,0.994194,0.598375,0.688684,0.606779,0.604890,0.603146,0.605008
3,0.999954,0.999937,0.999989,1.000000,0.999919,0.999899,0.999979,0.999986,0.998424,0.998469,0.998389,0.998290,0.998432,0.998477,0.998430,0.998323,0.994400,0.597290,0.688362,0.605732,0.603817,0.602067,0.603955
4,0.999986,0.999971,0.999912,0.999919,1.000000,0.999984,0.999952,0.999954,0.998294,0.998340,0.998140,0.998034,0.998365,0.998412,0.998243,0.998129,0.994058,0.595620,0.689018,0.604099,0.602192,0.600405,0.602285
5,0.999969,0.999986,0.999892,0.999899,0.999984,1.000000,0.999933,0.999936,0.998180,0.998260,0.998026,0.997919,0.998252,0.998333,0.998129,0.998015,0.993855,0.595010,0.688630,0.603532,0.601632,0.599837,0.601711
6,0.999964,0.999946,0.999986,0.999979,0.999952,0.999933,1.000000,0.999989,0.998289,0.998333,0.998240,0.998121,0.998328,0.998372,0.998311,0.998184,0.994098,0.597393,0.688343,0.605831,0.603940,0.602175,0.604040
7,0.999961,0.999945,0.999971,0.999986,0.999954,0.999936,0.999989,1.000000,0.998393,0.998440,0.998332,0.998237,0.998438,0.998484,0.998409,0.998306,0.994302,0.596303,0.688006,0.604781,0.602863,0.601092,0.602983
8,0.998347,0.998237,0.998316,0.998424,0.998294,0.998180,0.998289,0.998393,1.000000,0.999981,0.999939,0.999940,0.999976,0.999956,0.999950,0.999945,0.997983,0.599775,0.686612,0.608760,0.606668,0.604779,0.606833
9,0.998392,0.998315,0.998359,0.998469,0.998340,0.998260,0.998333,0.998440,0.999981,1.000000,0.999922,0.999924,0.999958,0.999976,0.999933,0.999929,0.997895,0.599343,0.686093,0.608388,0.606301,0.604399,0.606448


In [20]:
pd.DataFrame(cosine_sim_list[3])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,1.000000,0.998468,0.996144,0.997643,0.994167,0.994455,0.991342,0.991940,0.986695,0.983356,0.977749,0.980964,0.981901,0.981158,0.974348,0.976469,0.906758,0.716972,0.494158,0.843364,0.828000,0.846291,0.861725
1,0.998468,1.000000,0.995653,0.997588,0.990895,0.994240,0.988914,0.990043,0.985589,0.986153,0.977303,0.981101,0.978969,0.982111,0.971919,0.974711,0.907722,0.713033,0.494413,0.842449,0.826663,0.847333,0.863137
2,0.996144,0.995653,1.000000,0.997074,0.988452,0.989943,0.993628,0.989543,0.984633,0.982204,0.985421,0.982032,0.977790,0.978170,0.980377,0.975613,0.909979,0.715398,0.492948,0.840729,0.831067,0.851967,0.862708
3,0.997643,0.997588,0.997074,1.000000,0.991508,0.993320,0.991780,0.993905,0.986566,0.985017,0.980923,0.986230,0.981013,0.982118,0.976592,0.980893,0.910108,0.714248,0.498747,0.841627,0.824213,0.843774,0.860933
4,0.994167,0.990895,0.988452,0.991508,1.000000,0.998443,0.995779,0.997701,0.981298,0.975400,0.969753,0.975359,0.987559,0.984267,0.977691,0.982015,0.894407,0.710777,0.485492,0.855922,0.838263,0.840204,0.857777
5,0.994455,0.994240,0.989943,0.993320,0.998443,1.000000,0.995239,0.997573,0.981913,0.979877,0.971176,0.977187,0.986214,0.986831,0.977009,0.981819,0.897431,0.707952,0.485960,0.856282,0.838328,0.842835,0.860677
6,0.991342,0.988914,0.993628,0.991780,0.995779,0.995239,1.000000,0.996687,0.980168,0.974841,0.978841,0.977114,0.984844,0.982365,0.985752,0.982387,0.898178,0.709642,0.483729,0.854988,0.843608,0.847633,0.860060
7,0.991940,0.990043,0.989543,0.993905,0.997701,0.997573,0.996687,1.000000,0.981203,0.976937,0.973082,0.980602,0.986955,0.985374,0.980413,0.986727,0.897435,0.707839,0.489636,0.854850,0.835247,0.838208,0.857406
8,0.986695,0.985589,0.984633,0.986566,0.981298,0.981913,0.980168,0.981203,1.000000,0.996968,0.992368,0.995622,0.994403,0.994093,0.988275,0.990379,0.918222,0.713108,0.491847,0.830128,0.814224,0.829966,0.846061
9,0.983356,0.986153,0.982204,0.985017,0.975400,0.979877,0.974841,0.976937,0.996968,1.000000,0.989583,0.994154,0.988777,0.994380,0.982601,0.986199,0.920117,0.711924,0.495364,0.827733,0.811024,0.829967,0.846729


In [21]:
pd.DataFrame(cosine_sim_list[4])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,1.000000,0.999915,0.999703,0.999777,0.999765,0.999732,0.999580,0.999596,0.996821,0.996571,0.996641,0.996674,0.996836,0.996635,0.996772,0.996744,0.993514,0.522173,0.405521,0.578677,0.575148,0.568856,0.572146
1,0.999915,1.000000,0.999649,0.999760,0.999627,0.999767,0.999474,0.999527,0.996908,0.996854,0.996750,0.996830,0.996851,0.996852,0.996811,0.996831,0.993749,0.520698,0.406521,0.577021,0.573448,0.567209,0.570543
2,0.999703,0.999649,1.000000,0.999942,0.999356,0.999353,0.999771,0.999655,0.996364,0.996157,0.996883,0.996764,0.996257,0.996096,0.996904,0.996722,0.993291,0.526066,0.408974,0.581361,0.577961,0.571651,0.574812
3,0.999777,0.999760,0.999942,1.000000,0.999486,0.999519,0.999768,0.999767,0.996451,0.996283,0.996806,0.996821,0.996409,0.996289,0.996890,0.996843,0.993436,0.522454,0.406692,0.578470,0.574946,0.568607,0.571889
4,0.999765,0.999627,0.999356,0.999486,1.000000,0.999912,0.999697,0.999773,0.996271,0.995967,0.995968,0.996056,0.996912,0.996655,0.996718,0.996747,0.993256,0.516514,0.398779,0.574448,0.570892,0.564256,0.567566
5,0.999732,0.999767,0.999353,0.999519,0.999912,1.000000,0.999640,0.999752,0.996424,0.996320,0.996142,0.996277,0.996991,0.996941,0.996819,0.996897,0.993557,0.515095,0.399832,0.572842,0.569242,0.562663,0.566018
6,0.999580,0.999474,0.999771,0.999768,0.999697,0.999640,1.000000,0.999942,0.995928,0.995668,0.996331,0.996265,0.996440,0.996224,0.996963,0.996837,0.993151,0.520543,0.402465,0.577233,0.573806,0.567157,0.570338
7,0.999596,0.999527,0.999655,0.999767,0.999773,0.999752,0.999942,1.000000,0.995958,0.995738,0.996196,0.996265,0.996539,0.996364,0.996896,0.996906,0.993238,0.516884,0.400099,0.574302,0.570752,0.564072,0.567375
8,0.996821,0.996908,0.996364,0.996451,0.996271,0.996424,0.995928,0.995958,1.000000,0.999882,0.999570,0.999688,0.999561,0.999509,0.999254,0.999310,0.996244,0.531202,0.428780,0.583602,0.580088,0.574717,0.578002
9,0.996571,0.996854,0.996157,0.996283,0.995967,0.996320,0.995668,0.995738,0.999882,1.000000,0.999491,0.999659,0.999373,0.999565,0.999107,0.999214,0.996231,0.529481,0.429063,0.581839,0.578276,0.572949,0.576283


In [22]:
pd.DataFrame(cosine_sim_list[5])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,1.000000,0.999975,0.999921,0.999939,0.999970,0.999956,0.999931,0.999933,0.998201,0.998270,0.998202,0.998201,0.998316,0.998408,0.998383,0.998357,0.993899,0.443543,0.445543,0.472662,0.470114,0.465032,0.467590
1,0.999975,1.000000,0.999910,0.999945,0.999934,0.999969,0.999909,0.999927,0.998114,0.998244,0.998130,0.998149,0.998214,0.998368,0.998297,0.998291,0.993721,0.443037,0.444623,0.472244,0.469681,0.464596,0.467168
2,0.999921,0.999910,1.000000,0.999976,0.999850,0.999850,0.999970,0.999929,0.998026,0.998111,0.998215,0.998161,0.998088,0.998195,0.998344,0.998265,0.993678,0.445603,0.445124,0.474604,0.472116,0.467095,0.469592
3,0.999939,0.999945,0.999976,1.000000,0.999885,0.999901,0.999962,0.999969,0.998046,0.998153,0.998183,0.998190,0.998130,0.998260,0.998334,0.998316,0.993730,0.443716,0.444933,0.472833,0.470283,0.465237,0.467795
4,0.999970,0.999934,0.999850,0.999885,1.000000,0.999975,0.999921,0.999939,0.998059,0.998114,0.998013,0.998031,0.998254,0.998333,0.998275,0.998268,0.993657,0.441456,0.445417,0.470590,0.468023,0.462915,0.465491
5,0.999956,0.999969,0.999850,0.999901,0.999975,1.000000,0.999910,0.999944,0.997982,0.998097,0.997950,0.997987,0.998162,0.998303,0.998198,0.998211,0.993485,0.440922,0.444496,0.470143,0.467562,0.462450,0.465041
6,0.999931,0.999909,0.999970,0.999962,0.999921,0.999910,1.000000,0.999976,0.997922,0.997992,0.998064,0.998029,0.998064,0.998158,0.998274,0.998214,0.993472,0.443486,0.445021,0.472505,0.469998,0.464949,0.467466
7,0.999933,0.999927,0.999929,0.999969,0.999939,0.999944,0.999976,1.000000,0.997926,0.998018,0.998016,0.998041,0.998090,0.998207,0.998249,0.998250,0.993508,0.441590,0.444801,0.470724,0.468156,0.463082,0.465659
8,0.998201,0.998114,0.998026,0.998046,0.998059,0.997982,0.997922,0.997926,1.000000,0.999961,0.999886,0.999913,0.999945,0.999922,0.999891,0.999891,0.997855,0.444126,0.440372,0.474109,0.471462,0.466172,0.468827
9,0.998270,0.998244,0.998111,0.998153,0.998114,0.998097,0.997992,0.998018,0.999961,1.000000,0.999865,0.999919,0.999888,0.999944,0.999853,0.999880,0.997705,0.443680,0.439112,0.473791,0.471125,0.465821,0.468494


## Analysing the predictions ##

In [23]:
def get_predictions(model, input):
    with torch.no_grad():
        tokenized_inp = model.tokenizer(input, padding=True, truncation=True, return_tensors='pt').to(device)
        encoder_out = model.transformer_encoder(**tokenized_inp)
        logits = model.linear_classifier(encoder_out.last_hidden_state[:, 0, :])
        return logits.cpu().detach()

In [24]:
def analyse_predictions(prediction_list, sentence_list):
    cols = ['Sentence', 'Best label', 'Best label score', 'Positive label']
    values = list()
    for (report, pred) in zip(sentence_list, prediction_list.numpy()):
        best_label = labels_subset[pred.argmax()]
        best_label_score = pred.max()
        positive_label = ','.join([labels_subset[i] for i, e in enumerate(pred) if e > 0.0])
        values.append([report, best_label, best_label_score, positive_label])
    return pd.DataFrame(values, columns=cols)

In [55]:
predictions_list = [get_predictions(radbert_multi_model, all_sentence_list) for radbert_multi_model in radbert_multi_models]

In [56]:
print(predictions_list)
print('\t'.join([str(predictions.shape) for predictions in predictions_list]))

[tensor([[-1.7488,  0.5670, -0.4445,  ..., -2.1964, -2.7806, -1.4778],
        [-1.7348,  0.5451, -0.4546,  ..., -2.2575, -2.8312, -1.5237],
        [-1.7223,  0.5234, -0.4772,  ..., -2.2431, -2.8114, -1.4929],
        ...,
        [-2.4218,  0.7410, -0.3830,  ..., -2.4628, -2.9541, -1.5499],
        [-2.4044,  0.5986, -0.2292,  ..., -2.5301, -3.0473, -1.6426],
        [-2.3924,  0.5881, -0.2309,  ..., -2.4989, -3.0262, -1.6263]]), tensor([[-1.1170,  0.1777, -0.7489,  ..., -2.6124, -2.8236, -0.3782],
        [-1.1078,  0.1651, -0.7595,  ..., -2.6320, -2.8361, -0.3883],
        [-1.1106,  0.1714, -0.7718,  ..., -2.6230, -2.8401, -0.3805],
        ...,
        [-2.7658,  1.2113, -0.7962,  ..., -3.4114, -3.7316, -1.7808],
        [-2.7541,  1.1079, -0.7758,  ..., -3.4620, -3.7784, -1.8494],
        [-2.7420,  1.0787, -0.7920,  ..., -3.4644, -3.7836, -1.8523]]), tensor([[-1.8971, -0.0314, -0.7090,  ..., -2.9956, -3.1069, -0.2900],
        [-1.8948, -0.0241, -0.7120,  ..., -2.9972, -3.1068,

In [57]:
analyse_predictions(predictions_list[0], all_sentence_list)

,Sentence,Best label,Best label score,Positive label
0,A small pleural effusion can be seen in the report in the upper-left part,opacity,0.567028,"opacity,opacity-left"
1,A small pleural effusion can be seen in the report in the lower-left part,opacity,0.545061,"opacity,opacity-left"
2,A small pleural effusion can be seen in the report in the right-sided part,opacity,0.523431,"opacity,opacity-left"
3,A small pleural effusion can be seen in the report in the left-sided part,opacity,0.505500,"opacity,opacity-left"
4,A large pleural effusion can be seen in the report in the upper-left part,opacity,0.643405,"opacity,opacity-left"
5,A large pleural effusion can be seen in the report in the lower-left part,opacity,0.626830,"opacity,opacity-left"
6,A large pleural effusion can be seen in the report in the right-sided part,opacity,0.605754,"opacity,opacity-left"
7,A large pleural effusion can be seen in the report in the left-sided part,opacity,0.588651,"opacity,opacity-left"
8,The report shows a small upper-left pleural effusion,opacity,0.442813,"opacity,opacity-left"
9,The report shows a small lower-left pleural effusion,opacity,0.410573,"opacity,opacity-left"


In [58]:
analyse_predictions(predictions_list[1], all_sentence_list)

,Sentence,Best label,Best label score,Positive label
0,A small pleural effusion can be seen in the report in the upper-left part,effusion,0.866979,"opacity,bluntedcp,effusion,peffusion,bluntedcp-left,peffusion-right"
1,A small pleural effusion can be seen in the report in the lower-left part,effusion,0.854406,"opacity,bluntedcp,effusion,peffusion,bluntedcp-left,peffusion-right"
2,A small pleural effusion can be seen in the report in the right-sided part,effusion,0.852497,"opacity,bluntedcp,effusion,peffusion,bluntedcp-left,peffusion-right"
3,A small pleural effusion can be seen in the report in the left-sided part,effusion,0.854425,"opacity,bluntedcp,effusion,peffusion,bluntedcp-left,peffusion-right"
4,A large pleural effusion can be seen in the report in the upper-left part,effusion,0.882989,"opacity,bluntedcp,effusion,peffusion,bluntedcp-left,peffusion-right"
5,A large pleural effusion can be seen in the report in the lower-left part,effusion,0.871430,"opacity,bluntedcp,effusion,peffusion,bluntedcp-left,peffusion-right"
6,A large pleural effusion can be seen in the report in the right-sided part,effusion,0.868850,"opacity,bluntedcp,effusion,peffusion,bluntedcp-left,peffusion-right"
7,A large pleural effusion can be seen in the report in the left-sided part,effusion,0.870863,"opacity,bluntedcp,effusion,peffusion,bluntedcp-left,peffusion-right"
8,The report shows a small upper-left pleural effusion,effusion,0.908669,"opacity,bluntedcp,effusion,peffusion,bluntedcp-left,peffusion-right"
9,The report shows a small lower-left pleural effusion,effusion,0.901187,"opacity,bluntedcp,effusion,peffusion,bluntedcp-left,peffusion-right"


In [59]:
analyse_predictions(predictions_list[2], all_sentence_list)

,Sentence,Best label,Best label score,Positive label
0,A small pleural effusion can be seen in the report in the upper-left part,effusion,1.547613,"bluntedcp,effusion,peffusion,bluntedcp-left,peffusion-left,peffusion-right"
1,A small pleural effusion can be seen in the report in the lower-left part,effusion,1.549599,"bluntedcp,effusion,peffusion,bluntedcp-left,peffusion-left,peffusion-right"
2,A small pleural effusion can be seen in the report in the right-sided part,effusion,1.558709,"bluntedcp,effusion,peffusion,bluntedcp-left,peffusion-left,peffusion-right"
3,A small pleural effusion can be seen in the report in the left-sided part,effusion,1.561236,"bluntedcp,effusion,peffusion,bluntedcp-left,peffusion-left,peffusion-right"
4,A large pleural effusion can be seen in the report in the upper-left part,effusion,1.546569,"bluntedcp,effusion,peffusion,bluntedcp-left,peffusion-left,peffusion-right"
5,A large pleural effusion can be seen in the report in the lower-left part,effusion,1.548514,"bluntedcp,effusion,peffusion,bluntedcp-left,peffusion-left,peffusion-right"
6,A large pleural effusion can be seen in the report in the right-sided part,effusion,1.557829,"bluntedcp,effusion,peffusion,bluntedcp-left,peffusion-left,peffusion-right"
7,A large pleural effusion can be seen in the report in the left-sided part,effusion,1.560500,"bluntedcp,effusion,peffusion,bluntedcp-left,peffusion-left,peffusion-right"
8,The report shows a small upper-left pleural effusion,effusion,1.571355,"bluntedcp,effusion,peffusion,bluntedcp-left,peffusion-left,peffusion-right"
9,The report shows a small lower-left pleural effusion,effusion,1.575403,"bluntedcp,effusion,peffusion,bluntedcp-left,peffusion-left,peffusion-right"


In [60]:
analyse_predictions(predictions_list[3], all_sentence_list)

,Sentence,Best label,Best label score,Positive label
0,A small pleural effusion can be seen in the report in the upper-left part,opacity,0.613540,"opacity,effusion,opacity-left"
1,A small pleural effusion can be seen in the report in the lower-left part,opacity,0.592346,"opacity,effusion,opacity-left"
2,A small pleural effusion can be seen in the report in the right-sided part,opacity,0.572003,"opacity,opacity-left"
3,A small pleural effusion can be seen in the report in the left-sided part,opacity,0.521230,"opacity,effusion,opacity-left"
4,A large pleural effusion can be seen in the report in the upper-left part,opacity,0.729843,"opacity,effusion,opacity-left"
5,A large pleural effusion can be seen in the report in the lower-left part,opacity,0.709358,"opacity,effusion,opacity-left"
6,A large pleural effusion can be seen in the report in the right-sided part,opacity,0.699149,"opacity,effusion,opacity-left"
7,A large pleural effusion can be seen in the report in the left-sided part,opacity,0.641070,"opacity,effusion,opacity-left"
8,The report shows a small upper-left pleural effusion,opacity,0.463092,"opacity,effusion,opacity-left"
9,The report shows a small lower-left pleural effusion,opacity,0.417688,"opacity,effusion,opacity-left"


In [61]:
analyse_predictions(predictions_list[4], all_sentence_list)

,Sentence,Best label,Best label score,Positive label
0,A small pleural effusion can be seen in the report in the upper-left part,effusion,1.300664,"opacity,bronchialdilation,bluntedcp,effusion,peffusion,opacity-left,bluntedcp-left,peffusion-left,peffusion-right,bluntedcp-2,bluntedcp-5"
1,A small pleural effusion can be seen in the report in the lower-left part,effusion,1.288970,"opacity,bronchialdilation,bluntedcp,effusion,peffusion,opacity-left,bluntedcp-left,peffusion-left,peffusion-right,bluntedcp-2,bluntedcp-5"
2,A small pleural effusion can be seen in the report in the right-sided part,effusion,1.284592,"opacity,bronchialdilation,bluntedcp,effusion,peffusion,opacity-left,bluntedcp-left,peffusion-left,peffusion-right,bluntedcp-2,bluntedcp-5"
3,A small pleural effusion can be seen in the report in the left-sided part,effusion,1.295445,"opacity,bronchialdilation,bluntedcp,effusion,peffusion,opacity-left,bluntedcp-left,peffusion-left,peffusion-right,bluntedcp-2,bluntedcp-5"
4,A large pleural effusion can be seen in the report in the upper-left part,effusion,1.332642,"opacity,bronchialdilation,bluntedcp,effusion,peffusion,opacity-left,bluntedcp-left,bluntedcp-right,peffusion-left,peffusion-right,bluntedcp-2,bluntedcp-5"
5,A large pleural effusion can be seen in the report in the lower-left part,effusion,1.320688,"opacity,bronchialdilation,bluntedcp,effusion,peffusion,opacity-left,bluntedcp-left,bluntedcp-right,peffusion-left,peffusion-right,bluntedcp-2,bluntedcp-5"
6,A large pleural effusion can be seen in the report in the right-sided part,effusion,1.315943,"opacity,bronchialdilation,bluntedcp,effusion,peffusion,opacity-left,bluntedcp-left,peffusion-left,peffusion-right,bluntedcp-2,bluntedcp-5"
7,A large pleural effusion can be seen in the report in the left-sided part,effusion,1.326762,"opacity,bronchialdilation,bluntedcp,effusion,peffusion,opacity-left,bluntedcp-left,bluntedcp-right,peffusion-left,peffusion-right,bluntedcp-2,bluntedcp-5"
8,The report shows a small upper-left pleural effusion,effusion,1.127411,"opacity,bronchialdilation,bluntedcp,effusion,peffusion,opacity-left,bluntedcp-left,peffusion-left,peffusion-right,bluntedcp-5"
9,The report shows a small lower-left pleural effusion,effusion,1.117491,"opacity,bronchialdilation,bluntedcp,effusion,peffusion,opacity-left,bluntedcp-left,peffusion-left,peffusion-right,bluntedcp-5"


In [62]:
analyse_predictions(predictions_list[5], all_sentence_list)

,Sentence,Best label,Best label score,Positive label
0,A small pleural effusion can be seen in the report in the upper-left part,effusion,2.003458,"opacity,bluntedcp,effusion,peffusion,bluntedcp-left,bluntedcp-right,peffusion-left,peffusion-right,bluntedcp-2,bluntedcp-5"
1,A small pleural effusion can be seen in the report in the lower-left part,effusion,2.007624,"opacity,bluntedcp,effusion,peffusion,bluntedcp-left,bluntedcp-right,peffusion-left,peffusion-right,bluntedcp-2,bluntedcp-5"
2,A small pleural effusion can be seen in the report in the right-sided part,effusion,2.010264,"opacity,bluntedcp,effusion,peffusion,bluntedcp-left,bluntedcp-right,peffusion-left,peffusion-right,bluntedcp-2,bluntedcp-5"
3,A small pleural effusion can be seen in the report in the left-sided part,effusion,2.009854,"opacity,bluntedcp,effusion,peffusion,bluntedcp-left,bluntedcp-right,peffusion-left,peffusion-right,bluntedcp-2,bluntedcp-5"
4,A large pleural effusion can be seen in the report in the upper-left part,effusion,1.999502,"opacity,bluntedcp,effusion,peffusion,bluntedcp-left,bluntedcp-right,peffusion-left,peffusion-right,bluntedcp-2,bluntedcp-5"
5,A large pleural effusion can be seen in the report in the lower-left part,effusion,2.003667,"opacity,bluntedcp,effusion,peffusion,bluntedcp-left,bluntedcp-right,peffusion-left,peffusion-right,bluntedcp-2,bluntedcp-5"
6,A large pleural effusion can be seen in the report in the right-sided part,effusion,2.006302,"opacity,bluntedcp,effusion,peffusion,bluntedcp-left,bluntedcp-right,peffusion-left,peffusion-right,bluntedcp-2,bluntedcp-5"
7,A large pleural effusion can be seen in the report in the left-sided part,effusion,2.006024,"opacity,bluntedcp,effusion,peffusion,bluntedcp-left,bluntedcp-right,peffusion-left,peffusion-right,bluntedcp-2,bluntedcp-5"
8,The report shows a small upper-left pleural effusion,effusion,2.053808,"opacity,bluntedcp,effusion,peffusion,bluntedcp-left,bluntedcp-right,peffusion-left,peffusion-right,bluntedcp-2,bluntedcp-5"
9,The report shows a small lower-left pleural effusion,effusion,2.059954,"opacity,bluntedcp,effusion,peffusion,bluntedcp-left,bluntedcp-right,peffusion-left,peffusion-right,bluntedcp-2,bluntedcp-5"


## Bigger reports ##

In [34]:
dataset1_template = """xr- chest pa  view
findings
lungs: normal.
trachea: normal.
carina: normal.
right and left main bronchi: normal.
pleura: normal.
heart: normal.
right heart border: normal.
left heart border: normal.
pulmonary bay: normal.
pulmonary hila: normal.
aorta: normal.
thoracic spine: normal.
other visualized bones: normal.
visualized soft tissues: normal.
diaphragm: normal.
visualized abdomen:  normal.
visualized neck: normal.
"""

dataset1_pleural_issue="""xr- chest pa view
findings
lungs: normal.
trachea: normal.
carina: normal.
right and left main bronchi: normal.
pleura: left costophrenic angle is blunted with thin stripe of homogenous opacity along left lateral chest wall.
heart: normal.
right heart border: normal.
left heart border: normal.
pulmonary bay: normal.
pulmonary hila: normal.
aorta: normal.
thoracic spine: normal.
other visualized bones: normal.
visualized soft tissues: normal.
diaphragm: normal.
visualized abdomen:  normal.
visualized neck: normal.
"""

dataset2_template="""6191206|3862169|x-ray chest pa/ap view of 09-feb-2018:
results:
post cabg status.
no focal lesion seen in the lung parenchyma.
cp angles and domes of the diaphragm are normal.
both hila are normal. pulmonary vasculature is normal.
cardiac size and configuration is normal.
trachea is central; no mediastinal shift is seen.
bony thorax and soft tissues of the chest wall are normal.
impression: no abnormality detected in the view obtained.
"""

dataset3_template="""x-ray chest (pa view)
the cardio thoracic ratio is normal.
the heart size and configuration are within normal limits.
the aortic arch is normal.
the lung fields show normal broncho-vascular markings.
both the pulmonary hila are normal in size.
the costophrenic and cardiophrenic recesses and the domes of
diaphragm are normal.
the bones and soft tissues of the chest wall show no abnormality.
impression : normal study.
dr.shakthi kumar
radiologist
ss
________________________________________________________
"""

In [40]:
print(len(radbert_multi_models[0].tokenizer(dataset1_pleural_issue).input_ids))

160


In [35]:
all_embeddings_list2 = [get_sentence_embeddings(radbert_multi_model, [dataset1_template, dataset1_pleural_issue, dataset2_template, dataset3_template]) for radbert_multi_model in radbert_multi_models]

In [36]:
print(all_embeddings_list2)
print('\t'.join([str(all_embeddings2.shape) for all_embeddings2 in all_embeddings_list2]))

[tensor([[ 0.3264,  0.3047,  0.3965,  ..., -1.9097,  0.1234, -0.8253],
        [ 0.3064, -0.2484, -0.1224,  ..., -2.5580, -0.1053, -0.2105],
        [ 0.3933,  0.0246,  0.2106,  ..., -1.8008,  0.1709, -0.9187],
        [ 0.6743,  0.4189,  0.3100,  ..., -1.8266,  0.4676, -0.8214]], device='cuda:0'), tensor([[ 0.2907,  0.3742,  0.1757,  ..., -1.5937,  0.2902, -0.4703],
        [ 0.1481, -0.4444, -0.3137,  ..., -2.3469, -0.1217,  0.0116],
        [ 0.2351,  0.2122,  0.0247,  ..., -1.7078,  0.3107, -0.4207],
        [ 0.5779,  0.5084, -0.1165,  ..., -1.3636,  0.4281, -0.4336]], device='cuda:0'), tensor([[ 5.3438e-04,  1.8902e-01,  2.3029e-02,  ..., -1.5836e+00,  1.1809e-01, -3.4489e-01],
        [-5.0564e-02, -4.1922e-01, -3.0094e-01,  ..., -2.1360e+00, -1.7312e-02, -2.3645e-01],
        [-4.1308e-02,  9.8564e-02, -6.1162e-02,  ..., -1.7547e+00,  1.2441e-01, -4.2596e-01],
        [ 3.5992e-02,  2.2589e-01, -1.0484e-01,  ..., -1.6222e+00,  2.8172e-01, -3.7828e-01]], device='cuda:0'), tensor

In [65]:
cosine_sim_list2 = [calc_cosine_sim_matrix(all_embeddings2) for all_embeddings2 in all_embeddings_list2]

In [66]:
pd.DataFrame(cosine_sim_list2[0])

,0,1,2,3
0,1.000000,0.658144,0.973873,0.882550
1,0.658144,1.000000,0.702644,0.698319
2,0.973873,0.702644,1.000000,0.873382
3,0.882550,0.698319,0.873382,1.000000


In [67]:
pd.DataFrame(cosine_sim_list2[1])

,0,1,2,3
0,1.000000,0.599835,0.982304,0.876604
1,0.599835,1.000000,0.650249,0.612620
2,0.982304,0.650249,1.000000,0.890336
3,0.876604,0.612620,0.890336,1.000000


In [68]:
pd.DataFrame(cosine_sim_list2[2])

,0,1,2,3
0,1.000000,0.818505,0.977808,0.924195
1,0.818505,1.000000,0.838402,0.835847
2,0.977808,0.838402,1.000000,0.936497
3,0.924195,0.835847,0.936497,1.000000


In [69]:
pd.DataFrame(cosine_sim_list2[3])

,0,1,2,3
0,1.000000,0.541230,0.973344,0.920404
1,0.541230,1.000000,0.588965,0.548456
2,0.973344,0.588965,1.000000,0.913254
3,0.920404,0.548456,0.913254,1.000000


In [70]:
pd.DataFrame(cosine_sim_list2[4])

,0,1,2,3
0,1.000000,0.527706,0.974638,0.878580
1,0.527706,1.000000,0.602073,0.568878
2,0.974638,0.602073,1.000000,0.893827
3,0.878580,0.568878,0.893827,1.000000


In [71]:
pd.DataFrame(cosine_sim_list2[5])

,0,1,2,3
0,1.000000,0.752108,0.979862,0.940965
1,0.752108,1.000000,0.764070,0.799732
2,0.979862,0.764070,1.000000,0.948634
3,0.940965,0.799732,0.948634,1.000000


In [72]:
predictions_list2 = [get_predictions(radbert_multi_model, [dataset1_template, dataset1_pleural_issue, dataset2_template, dataset3_template]) for radbert_multi_model in radbert_multi_models]

In [73]:
print(predictions_list2)

[tensor([[ 1.1145, -3.7944, -3.9405, -2.5833, -3.7998, -3.1079, -2.1409, -2.9018, -3.0312, -3.9262, -3.6631, -3.8038, -3.6908, -3.8595, -3.0626, -3.6263, -3.8737, -3.9619, -3.4211, -4.0856, -3.2126,
         -3.9956, -3.5138, -3.8792, -4.0674, -3.8545, -3.7203, -4.2172, -3.8095, -3.8922, -3.5546, -4.3026, -4.0035, -3.9933, -4.2730, -3.8059, -4.0118, -3.8330, -4.2458, -4.2581, -3.8282, -4.2079,
         -4.1373, -4.1535, -4.4707, -4.2371, -4.4445, -4.2050, -3.8641, -3.9413, -4.0732, -4.3452, -4.2605, -4.1134, -4.2707, -4.1763, -4.0340, -3.8950, -4.2504, -4.2983, -4.3631, -4.3919, -4.4036,
         -4.4184, -4.2708, -4.1729, -4.0480, -3.9906, -4.2335, -4.1273, -4.2487, -4.3224, -4.1333, -4.5137, -4.3449, -4.1361],
        [-2.4552, -0.2179, -0.7563, -2.6794, -1.1968, -2.5195, -1.4239, -2.2313, -2.3178, -1.8390, -2.0907, -2.1208, -1.7932, -2.2888, -2.4710, -2.3069, -1.7140, -2.0820, -2.1190, -2.6994, -2.6574,
         -2.2951, -2.2498, -2.4598, -2.5775, -2.7887, -2.8555, -2.7600, -2.9499,

In [43]:
analyse_predictions(predictions_list2[0], [dataset1_template, dataset1_pleural_issue, dataset2_template, dataset3_template])

,Sentence,Best label,Best label score,Positive label
0,xr- chest pa view\nfindings\nlungs: normal.\ntrachea: normal.\ncarina: normal.\nright and left main bronchi: normal.\npleura: normal.\nheart: normal.\nright heart border: normal.\nleft heart bord...,normal,2.325943,normal
1,\nxr- chest pa view\nfindings\nlungs: normal.\ntrachea: normal.\ncarina: normal.\nright and left main bronchi: normal.\npleura: left costophrenic angle is blunted with thin stripe of homogenous op...,opacity,-0.321856,
2,6191206|3862169|x-ray chest pa/ap view of 09-feb-2018:\nresults:\npost cabg status.\nno focal lesion seen in the lung parenchyma.\ncp angles and domes of the diaphragm are normal.\nboth hila are n...,normal,1.459038,normal
3,\nx-ray chest (pa view)\nthe cardio thoracic ratio is normal.\nthe heart size and configuration are within normal limits.\nthe aortic arch is normal.\nthe lung fields show normal broncho-vascular ...,normal,-0.069658,


In [74]:
analyse_predictions(predictions_list2[1], [dataset1_template, dataset1_pleural_issue, dataset2_template, dataset3_template])

,Sentence,Best label,Best label score,Positive label
0,xr- chest pa view\nfindings\nlungs: normal.\ntrachea: normal.\ncarina: normal.\nright and left main bronchi: normal.\npleura: normal.\nheart: normal.\nright heart border: normal.\nleft heart bord...,normal,2.325943,normal
1,\nxr- chest pa view\nfindings\nlungs: normal.\ntrachea: normal.\ncarina: normal.\nright and left main bronchi: normal.\npleura: left costophrenic angle is blunted with thin stripe of homogenous op...,opacity,-0.321856,
2,6191206|3862169|x-ray chest pa/ap view of 09-feb-2018:\nresults:\npost cabg status.\nno focal lesion seen in the lung parenchyma.\ncp angles and domes of the diaphragm are normal.\nboth hila are n...,normal,1.459038,normal
3,\nx-ray chest (pa view)\nthe cardio thoracic ratio is normal.\nthe heart size and configuration are within normal limits.\nthe aortic arch is normal.\nthe lung fields show normal broncho-vascular ...,normal,-0.069658,


In [75]:
analyse_predictions(predictions_list2[2], [dataset1_template, dataset1_pleural_issue, dataset2_template, dataset3_template])

,Sentence,Best label,Best label score,Positive label
0,xr- chest pa view\nfindings\nlungs: normal.\ntrachea: normal.\ncarina: normal.\nright and left main bronchi: normal.\npleura: normal.\nheart: normal.\nright heart border: normal.\nleft heart bord...,normal,2.145874,normal
1,\nxr- chest pa view\nfindings\nlungs: normal.\ntrachea: normal.\ncarina: normal.\nright and left main bronchi: normal.\npleura: left costophrenic angle is blunted with thin stripe of homogenous op...,normal,-0.687137,
2,6191206|3862169|x-ray chest pa/ap view of 09-feb-2018:\nresults:\npost cabg status.\nno focal lesion seen in the lung parenchyma.\ncp angles and domes of the diaphragm are normal.\nboth hila are n...,normal,1.114262,normal
3,\nx-ray chest (pa view)\nthe cardio thoracic ratio is normal.\nthe heart size and configuration are within normal limits.\nthe aortic arch is normal.\nthe lung fields show normal broncho-vascular ...,normal,0.153453,normal


In [76]:
analyse_predictions(predictions_list2[3], [dataset1_template, dataset1_pleural_issue, dataset2_template, dataset3_template])

,Sentence,Best label,Best label score,Positive label
0,xr- chest pa view\nfindings\nlungs: normal.\ntrachea: normal.\ncarina: normal.\nright and left main bronchi: normal.\npleura: normal.\nheart: normal.\nright heart border: normal.\nleft heart bord...,normal,1.379838,normal
1,\nxr- chest pa view\nfindings\nlungs: normal.\ntrachea: normal.\ncarina: normal.\nright and left main bronchi: normal.\npleura: left costophrenic angle is blunted with thin stripe of homogenous op...,opacity,0.405807,opacity
2,6191206|3862169|x-ray chest pa/ap view of 09-feb-2018:\nresults:\npost cabg status.\nno focal lesion seen in the lung parenchyma.\ncp angles and domes of the diaphragm are normal.\nboth hila are n...,normal,0.783583,normal
3,\nx-ray chest (pa view)\nthe cardio thoracic ratio is normal.\nthe heart size and configuration are within normal limits.\nthe aortic arch is normal.\nthe lung fields show normal broncho-vascular ...,normal,-0.062494,


In [77]:
analyse_predictions(predictions_list2[4], [dataset1_template, dataset1_pleural_issue, dataset2_template, dataset3_template])

,Sentence,Best label,Best label score,Positive label
0,xr- chest pa view\nfindings\nlungs: normal.\ntrachea: normal.\ncarina: normal.\nright and left main bronchi: normal.\npleura: normal.\nheart: normal.\nright heart border: normal.\nleft heart bord...,normal,1.774829,normal
1,\nxr- chest pa view\nfindings\nlungs: normal.\ntrachea: normal.\ncarina: normal.\nright and left main bronchi: normal.\npleura: left costophrenic angle is blunted with thin stripe of homogenous op...,opacity,1.158673,"opacity,bronchialdilation,opacity-left"
2,6191206|3862169|x-ray chest pa/ap view of 09-feb-2018:\nresults:\npost cabg status.\nno focal lesion seen in the lung parenchyma.\ncp angles and domes of the diaphragm are normal.\nboth hila are n...,normal,0.619379,normal
3,\nx-ray chest (pa view)\nthe cardio thoracic ratio is normal.\nthe heart size and configuration are within normal limits.\nthe aortic arch is normal.\nthe lung fields show normal broncho-vascular ...,normal,-0.923446,


In [78]:
analyse_predictions(predictions_list2[5], [dataset1_template, dataset1_pleural_issue, dataset2_template, dataset3_template])

,Sentence,Best label,Best label score,Positive label
0,xr- chest pa view\nfindings\nlungs: normal.\ntrachea: normal.\ncarina: normal.\nright and left main bronchi: normal.\npleura: normal.\nheart: normal.\nright heart border: normal.\nleft heart bord...,normal,2.295446,normal
1,\nxr- chest pa view\nfindings\nlungs: normal.\ntrachea: normal.\ncarina: normal.\nright and left main bronchi: normal.\npleura: left costophrenic angle is blunted with thin stripe of homogenous op...,opacity,-0.945140,
2,6191206|3862169|x-ray chest pa/ap view of 09-feb-2018:\nresults:\npost cabg status.\nno focal lesion seen in the lung parenchyma.\ncp angles and domes of the diaphragm are normal.\nboth hila are n...,normal,1.467981,normal
3,\nx-ray chest (pa view)\nthe cardio thoracic ratio is normal.\nthe heart size and configuration are within normal limits.\nthe aortic arch is normal.\nthe lung fields show normal broncho-vascular ...,normal,0.281569,normal
